In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras
import keras
from tensorflow.keras.datasets import mnist    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical    
import random

np.random.seed(0)

(X_train, y_train), (X_test, y_test) = mnist.load_data()    
# print(X_train.shape)
# print(X_test.shape)
# print(y_train)   

assert X_train.shape[0] == y_train.shape[0], "number of image is not equal to the number of labels"
assert X_test.shape[0] == y_test.shape[0], "number of image is not equal to the number of labels"
assert X_train.shape[1:] == (28,28), "dimension of the images are not 28x28"
assert X_test.shape[1:] == (28,28), "dimension of the images are not 28x28"

num_of_samples = []

cols = 5    
num_classes = 10   

fig, axs = plt.subplots(nrows = num_classes, ncols = cols, figsize = (5, 10))
fig.tight_layout()

for i in range(cols):
  for j in range(num_classes):
    x_selected = X_train[y_train == j]   
    axs[j][i].imshow(x_selected[random.randint(0, len(x_selected - 1)), :, :], cmap = plt.get_cmap("gray"))   

    
    if i == 2:
      axs[j][i].set_title(str(j))
      num_of_samples.append(len(x_selected))

print(num_of_samples)
plt.figure(figsize= (12, 4))
plt.bar(range(0, num_classes), num_of_samples)  
plt.title("Distribution of training dataset")
plt.xlabel("Class number")
plt.ylabel("Number of images")

"""one hot encoding of labels """
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

"""normalisation of all dataset
to categorise pixels from 0 to 1 by dividing with 255
255/255 = 1, 0/255 = 1, all other b/w 0 to 1
"""

X_train = X_train/255
X_test = X_test/255

"""flattening each image from 28x28 pixels 2D array to 784 pixels 1D array """
num_pixels = 784
X_train = X_train.reshape(X_train.shape[0], num_pixels)
X_test = X_test.reshape(X_test.shape[0], num_pixels)

def create_model():
  model = Sequential()
  model.add(Dense(10, input_dim = num_pixels, activation= "relu"))   #1st hidden layer with 784 input nodes
  model.add(Dense(10, activation="relu"))     #2nd hidden layer
  model.add(Dense(num_classes, activation= "softmax"))   #output layer
  model.compile(Adam(lr=0.01), loss="categorical_crossentropy", metrics=["accuracy"])
  return model

model = create_model()
print(model.summary())

history = model.fit(X_train, y_train, validation_split= 0.1, epochs= 10, batch_size= 200, verbose= 1, shuffle= 1)   #validation data will be 10% of training set          

plt.plot(history.history["loss"])     
plt.plot(history.history["val_loss"])   
plt.legend(["loss", "val_loss"])
plt.title("Loss")
plt.xlabel("epochs")

plt.plot(history.history["accuracy"])     
plt.plot(history.history["val_accuracy"])   
plt.legend(["accuracy", "val_accuracy"])
plt.title("Accuracy")
plt.xlabel("epochs")


score = model.evaluate(X_test, y_test, verbose=0)
print(type(score))
print("TEST SCORE : ", score[0])
print("TEST ACCURACY : ", score[1])

In [ ]:
""" importing new image """
import requests
from PIL import Image     #Python image library

url = "https://www.nengo.ai/keras-lmu/v0.3.1/_images/examples_psMNIST_9_0.png"
response = requests.get(url, stream=True)
img = Image.open(response.raw)
plt.imshow(img)


import cv2

img_array = np.asarray(img)     #converts the input to array 
print(img_array.shape)
img_resized = cv2.resize(img_array, (28, 28))
image = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
print(image.shape)
# image = cv2.bitwise_not(image)     #to flip the background color with number color
plt.imshow(image, cmap = plt.get_cmap("gray"))

"""normalising and reshaping the image """
image = image/255
image = image.reshape(1, 784)


""" prediction """
prediction = np.argmax(model.predict(image), axis=-1)
print("Predicted Digit : ", prediction)